In [1]:
from elasticsearch import Elasticsearch
from datetime import datetime
es = Elasticsearch()

class tutorial:
    # create an instance of elasticsearch and assign it to port 9200
    ES_HOST = {"host": "localhost", "port": 9200}
    es = Elasticsearch(hosts=[ES_HOST])


    def create_index(index_name):
        """Functionality to create index."""
        resp = es.indices.create(index=index_name)
        print(resp)


    def document_add(index_name, doc_type, doc, doc_id=None):
        """Funtion to add a document by providing index_name,
        document type, document contents as doc and document id."""
        resp = es.index(index=index_name, doc_type=doc_type, body=doc, id=doc_id)
        #print(resp)


    def document_view(index_name, doc_type, doc_id):
        """Function to view document."""
        resp = es.get(index=index_name, doc_type=doc_type, id=doc_id)
        document = resp["_source"]['Post']
        print(document)


    def document_update(index_name, doc_type, doc_id, doc=None, new=None):
        """Function to edit a document either updating existing fields or adding a new field."""
        if doc:
            resp = es.index(index=index_name, doc_type=doc_type,
                            id=doc_id, body=doc)
            print(resp)
        else:
            resp = es.update(index=index_name, doc_type=doc_type,
                             id=doc_id, body={"doc": new})


    def document_delete(index_name, doc_type, doc_id):
        """Function to delete a specific document."""
        resp = es.delete(index=index_name, doc_type=doc_type, id=doc_id)
        print(resp)


    def delete_index(index_name):
        """Delete an index by specifying the index name"""
        resp = es.indices.delete(index=index_name)
        print(resp)

In [20]:
from elasticsearch import Elasticsearch
 
es = Elasticsearch()

search_qry = "jaws"

res = es.search(index="test_v3", body={"query": {"match": {'Post':search_qry}}})

print("%d documents found" % res['hits']['total'])
for doc in res['hits']['hits']:
    print("%s) %s" % (doc['_id'], doc['_source']))

713 documents found
QNDDLGoBSGrx7-wy1xrx) {'ID': '662', 'Title': 'accessing jaws homepage', 'Post': 'hello judith:           wat you call the jaws home page, is in fact, the  jaws window of jaws on your computer.           so, when you press, insert+j, from anywhere within  windows or any program, focus should be given to the jaws window of the  version of jfw you are using. when you use this command,   jaws should say:           jaws context menu, options submenu           home pages, are generally referred too as the  initial page of a web site.           hth           dave durber', 'Tag': 'Answer'}
-9DELGoBSGrx7-wyBB7H) {'ID': '3160', 'Title': 'control, alt and j', 'Post': '               i also had this problem.               just now, i killed jaws 17, and  i tried the alt+ctrl+j and jaws        17 came up. i did this three times.              perhaps there is an improvement with the latest jaws 17.              i use the shift+ctrl+j to bring up the last version of jaws. this    

In [18]:
from elasticsearch import Elasticsearch
from itertools import chain
from nltk.corpus import wordnet

SW_LIST = ['firefox', 'chrome', 'microsoft edge', 'opera', 'internet explorer', 'waterfox', 'microsoft office',
           'microsoft word', 'microsoft excel', 'microsoft powerpoint', 'browser']

def return_conversions_for_query(query):
    es = Elasticsearch()
    
#     sw_name = []
#     other_words = []
#     for word in query.split():
#         if word in SW_LIST:
#             sw_name.append(word)
#         else:
#             synonyms = wordnet.synsets(word)
#             lemmas = list(set(chain.from_iterable([w.lemma_names() for w in synonyms])))[:5]
#             other_words.extend(lemmas)
#             #other_words.extend(word)
    
#     sw_name = ' '.join(sw_name)
#     other_words = ' '.join(other_words)
    #print(sw_name)
    #print(other_words)
    res = es.search(index="test_v2", body={"query": {"match": {'Original_post': {"query": query,
                                                    }}}})
    
#     res = es.search(index="test", body={"query": {
#                                     'bool': {
#                                           'must': {
#                                             'match': {'Post': sw_name}
#                                           },
#                                           'should': {
#                                             'term': {'Post': other_words}
#                                           }
#                                         }}})
    
    print("%d documents found" % res['hits']['total'])
    
    conversation =[]
    
    #for doc in res['hits']['hits']:
    for doc in res['hits']['hits']:
        print("%s) %s" % (doc['_id'], doc['_source']))
        conversation.append(doc['_source'])
        
    print(len(conversation))
    return conversation

In [4]:
import pandas as pd

search_qry = "microsoft word problem"
res = return_conversions_for_query(search_qry)
print(len(res))

NameError: name 'return_conversions_for_query' is not defined

In [58]:
import collections
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def to_json(query):
    es = Elasticsearch()
    res = es.search(index="test_v2",  size=5000, body={"query": {"match": {'Original_post': {"query": query, }}}})
    
    conversation =[]
    for doc in res['hits']['hits']:
        conversation.append(doc['_source'])
    
    df_result = pd.DataFrame(conversation)
    #df_result.columns = ['id', 'post', 'tag', 'title']
    #df_result = df_result.sort_values(['title', 'tag'], ascending=[True, False])
    d = collections.defaultdict(list)
    o = collections.OrderedDict()
    #print(df_result.head())
    for index, row in df_result.iterrows():
        ind = row[0]
        post = row[1]
        tag = row[2]
        title = row[3]
        
        temp = {}
        temp["id"] = ind
        temp['post'] = post
        temp['tag'] = tag
        #print(index, title,)
        d[title].append(temp)
        o[title] = True

    n_items = collections.defaultdict(list)
    for i,k in enumerate(o.keys()):
        d[k] = sorted(d[k], key = lambda i: i['id'])
        d[k] = sorted(d[k], key = lambda i: i['tag'], reverse=True)
        n_items[k].append(d[k])
        if i == 21:
            break
    print(n_items)

In [59]:
to_json("google hangout use")

defaultdict(<class 'list'>, {'google hangouts on the computer using nvda': [[{'id': '12269', 'post': 'google hangouts on the computer using nvda " hello guys. i need some advice on how to use nvda with google hangouts. i have a feeling that this won\'t be easy. has anyone here used it successfully, more than once? if necessary, email me off list. @govsta thanks in advance for all assistance. "', 'tag': 'Help'}, {'id': '12272', 'post': '" yes, nvda+spacebar to to switch to focus mode and all of these keyboard commands will work. if in browse mode, use nvda+f2 to bypass next keypress. "', 'tag': 'Answer'}, {'id': '12274', 'post': '" if you use it with chrome and nvda, it\'s actually quite easy. i\'ve done it several times, and haven\'t had any issue. in fact, if you google the phrase google hangout shortcut keys there is a whole list of commands you can use while in your browser using hangouts. another way to get there is to disable browse mode within nvda, then press shift slash, which 

In [ ]:
df_result = pd.DataFrame(res)
print(len(df_result))

In [ ]:
df_result.head()

In [ ]:
#df_result = df_result.sort_values(['Title', 'ID'], ascending=[True, True])

In [ ]:
df_result.head()

In [ ]:
from collections import defaultdict

d = defaultdict(list)
print(d)

In [ ]:
def assign_to_dict(row):
    title = row[5]
    ind = row[2]
    post = row[3]
    tag = row[4]
    
    temp = {}
    temp["id"] = ind
    temp['post'] = post
    temp['tag'] = tag
    
    d[title].append(temp)

In [ ]:
df_result.apply(assign_to_dict, axis = 1)

print(d['about reading numbers in nvda 2017.2 and before nvdadevelopers and experts please help me.'])

In [ ]:
import json
with open('../FInal Version Code/Data/result.json', 'w') as fp:
    json.dump(d, fp)

In [ ]:
for k in d.keys():
    print(d[k])
    print('--')

In [ ]:
from platform import python_version

print(python_version())